# Ivestment Porfolio Management

## Load investments and print short report

In [ ]:
import sys
import os
import logging

logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)

# Give priority to local packages (not needed in case Robson was installed by pip)
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(''), '..')))

import investor

In [ ]:
# Use data from cache
me=investor.Investor("../investor_ui_config.yaml")

# Pass a refreshMap to force refresh data from the Internet
# me=investor.Investor(
#     "../investor_ui_config.yaml",
#     refreshMap=dict(zip(investor.Investor.domains,len(investor.Investor.domains)*[True]))
# )

In [ ]:
me.config

In [ ]:
print(me)

## Get list of investment instruments and currencies

In [ ]:
me.portfolio[0]['obj'].funds()

## Get compound fund from a few instruments

In [ ]:
me.exchange.currency='USD'

In [ ]:
fund=me.portfolio[0]['obj'].getFund(['TraderBot KuCoin'],currencyExchange=me.exchange)
fund

## Compute periodic report

Pass a list to `kpi` parameter to show only specific KPIs. Avaialble are:

**Source of all information**
* `investor.KPI.BALANCE`\
plain balance
* `investor.KPI.MOVEMENTS`\
money added and removed from fund on period

**Cumulative movements**
* `investor.KPI.SAVINGS`\
cumulative money added and removed from fund

**Rate of accumulated gains**
* `investor.KPI.BALANCE_OVER_SAVINGS`\
balance ➗ savings

**Pure gain, on the period or accumulated**
* `investor.KPI.GAINS`\
cumulative gains
* `investor.KPI.PERIOD_GAIN`\
gain on each period

**Normalization**
* `investor.KPI.SHARES`\
amount of shares
* `investor.KPI.SHARE_VALUE`\
value of a share

**Performance**
* `investor.KPI.RATE_RETURN`\
percentage change of share value

**KPIs related to external sources**
* `investor.KPI.BENCHMARK`\
raw value of the benchmark
* `investor.KPI.BENCHMARK_RATE_RETURN`\
variation of benchmark in relation to the last period
* `investor.KPI.BENCHMARK_EXCESS_RETURN`\
relation between RATE_RETURN and BENCHMARK_RATE_RETURN, on each period

### Month and Year

In [ ]:
fund.report(benchmark=me.benchmarks[8]['obj'])

### Quarter & Year

In [ ]:
fund.report('Q',benchmark=me.benchmarks[9]['obj'])

### Daily

In [ ]:
fund.report('D',benchmark=me.benchmarks[9]['obj'])

#### Plain daily gains

In [ ]:
fund.report('D',benchmark=me.benchmarks[9]['obj'],kpi=[investor.KPI.PERIOD_GAIN,investor.KPI.GAINS,investor.KPI.RATE_RETURN])

In [ ]:
fund.periodicReport('M',benchmark=me.benchmarks[8]['obj'])

------

In [ ]:
period=fund.periodicReport('D')
period

In [ ]:
macroPeriod=fund.periodicReport('W')
macroPeriod

In [ ]:
macroPeriod.index

In [ ]:
import pandas as pd

p=dict(
    period                     = 'D',
    periodLabel                = 'day',
    periodFormatter            = '%w·%a',

    macroPeriod                = 'W',
    macroPeriodLabel           = 'week',
    macroPeriodFormatter       = '%Y-w%U'
)

i=1

if i==0:
    line=period[:macroPeriod.index[i]]
    nPeriods=period[:macroPeriod.index[i]].shape[0]
else:
    currentRange=(
        (period.index > macroPeriod.index[i-1]) &
        (period.index <= macroPeriod.index[i])
    )
    line=period[currentRange]
    nPeriods=period[currentRange].shape[0]

line=(
    # Add a row label, as '2020' or '4·Thu'
    pd.concat([line], axis=1, keys=[macroPeriod.index[i]])

    # Rename the title for labels so we can join latter
    .rename_axis(['time','KPI'], axis='columns')

    # Convert index from full DateTimeIndex to something that can be matched
    # across macro-periods, as just '08·Aug'
    .assign(
        **{
            p['periodLabel']: (
                line.index.strftime(p['periodFormatter'])
                if 'periodFormatter' in p
                else range(1,nPeriods+1,1)
            ),
            'firstindex': 'periods'
        }
    )
    .set_index(['firstindex',p['periodLabel']])
)
    
# # Add a row label, as '2020'
# line=pd.concat([line], axis=1, keys=[macroPeriod.index[i]])

# # Rename the title for labels so we can join latter
# line.rename_axis(['time','KPI'], axis='columns', inplace=True)

# line=pd.concat([line], axis=0, keys=['periods'])

line

In [ ]:
p=dict(
    period                     = 'D',
    periodLabel                = 'day',
    periodFormatter            = '%w·%a',

    macroPeriod                = 'W',
    macroPeriodLabel           = 'week',
    macroPeriodFormatter       = '%Y-w%U'
)

import pandas as pd

report=None

# Break the periodic report in chunks equivalent to the summary report
for i in range(len(macroPeriod.index)):

    currentRange=None

    if i==0:
        line=period[:macroPeriod.index[i]]
        nPeriods=period[:macroPeriod.index[i]].shape[0]
    else:
        currentRange=(
            (period.index > macroPeriod.index[i-1]) &
            (period.index <= macroPeriod.index[i])
        )
        line=period[currentRange]
        nPeriods=period[currentRange].shape[0]

    # Add a row label, as '2020'
    line=pd.concat([line], axis=1, keys=[macroPeriod.index[i]])

    # Rename the title for labels so we can join latter
    line.rename_axis(['time','KPI'], axis='columns', inplace=True)

    # Convert index from full DateTimeIndex to something that can be matched
    # across macro-periods, as just '08·Aug'
    if 'periodFormatter' in p:
        line[p['periodLabel']]=line.index.strftime(p['periodFormatter'])
    else:
        line[p['periodLabel']]=range(1,nPeriods+1,1)
    line.set_index(p['periodLabel'], inplace=True)
    line=pd.concat([line], axis=0, keys=['periods'])

    # Add to main report transposing it into a true row (we were columns until now)
    if report is not None:
        report=pd.concat([report,line.T], sort=True)
    else:
        report=line.T

report

----

### Week & 4 Weeks
#### Performance report

In [ ]:
# fund=me.portfolio[0]['obj'].getFund(['ShiguBot Binance','ShiguBot MB'],currencyExchange=me.exchange)
fund=me.portfolio[0]['obj'].getFund(['TraderBot KuCoin'],currencyExchange=me.exchange)

fund.report(
    period='W',
    benchmark=me.benchmarks[8]['obj'],
    kpi=[
        investor.KPI.RATE_RETURN,
        investor.KPI.BENCHMARK_RATE_RETURN,
        investor.KPI.BENCHMARK_EXCESS_RETURN,
        investor.KPI.PERIOD_GAIN
    ],
)

#### Wealth Evolution

In [ ]:
fund.report(
    period='W',
    benchmark=me.benchmarks[9]['obj'],
    kpi=[
        investor.KPI.BALANCE,
        investor.KPI.BALANCE_OVER_SAVINGS,
        investor.KPI.GAINS,
        investor.KPI.SAVINGS,
        investor.KPI.MOVEMENTS
    ],
)

#### Plain weekly gains

In [ ]:
fund.report('W',benchmark=me.benchmarks[9]['obj'],kpi=[investor.KPI.PERIOD_GAIN])